<a href="https://colab.research.google.com/github/rakirs3333/DeepFakeDetection/blob/main/web_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=95d4c58d791f9c177692d22edf86a3209ff03aacad078d9091343be22894533b
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Setting up the Model and Server


In [ ]:
!ngrok authtoken 29vgslIIoPImj5ML3dS1DN84SZs_6ypNS4BAStsXVYrHshpbx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
cd drive/My Drive/webapp-deepfake

/content/drive/.shortcut-targets-by-id/1-7I9JIBPqA_GX6JYqMnwQBHjeY9S0dm4/deepfake


In [ ]:
!pip3 install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=c78acdd6eaf97e2a8bb351d3dcb6918381a42e9df2578e563b62b7af60f742b4
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import torch
import face_recognition
from torch import nn
from torchvision import models
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        for i,frame in enumerate(self.frame_extract(video_path)):
            faces = face_recognition.face_locations(frame)
            try:
              top,right,bottom,left = faces[0]
              frame = frame[top:bottom,left:right,:]
            except:
              pass
            frames.append(self.transform(frame))
            if(len(frames) == self.count):
              break
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)  ##$$
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path)
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image

In [ ]:
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return self.dp(self.linear1(x_lstm[:,-1,:]))

In [ ]:
sm = nn.Softmax()
def predict(model,img):
    logits = model(img.to('cuda'))
    logits = sm(logits)
    _,prediction = torch.max(logits,1)
    confidence = logits[:,int(prediction.item())].item()*100
    print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)
    return [int(prediction.item())]

In [ ]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])


model = Model(2).cuda()
path_to_model = '/content/drive/My Drive/webapp-deepfake/best.pt'
model.load_state_dict(torch.load(path_to_model))
model.eval()

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

Model(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os
app = Flask(__name__)
run_with_ngrok(app)
UPLOAD_FOLDER = "static"


@app.route("/", methods=['GET', 'POST'])  # website endpoint
def upload_predict():
    if request.method == 'POST':
        video_file = request.files['video']
        if video_file:
            video_location = os.path.join(UPLOAD_FOLDER, video_file.filename)
            video_file.save(video_location)
            path_to_videos= [video_location]
            print(path_to_videos)
            video_dataset = validation_dataset(path_to_videos,sequence_length = 40,transform = train_transforms)
            prediction = predict(model,video_dataset[0])
            if prediction[0] == 1:
              pred='REAL'
            else:
              pred='FAKE'
            return render_template("index.html", prediction=pred, video_loc=video_location)
    return render_template("index.html")

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://14ff-35-230-68-233.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Jun/2022 05:02:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:02:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:02:59] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:02:59] "GET /static/script.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:00] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:00] "GET /static/script.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:00] "GET /static/roadmap.jpg HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:00] "GET /static/deepfake.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:01] "GET /static/Flowchart.jpg HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2022 05:03:02] "GET /static/deepfake.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:02] "GET /static/Flowchart.jpg HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 05:03:02] "GET /static/roadmap.jpg HTTP/1.1" 200 -


In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(model,input_size=(4,40,3,112,112))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Layer (type:depth-idx)                        Output Shape              Param #
Model                                         [4, 2]                    --
├─Sequential: 1-1                             [160, 2048, 4, 4]         --
│    └─Conv2d: 2-1                            [160, 64, 56, 56]         9,408
│    └─BatchNorm2d: 2-2                       [160, 64, 56, 56]         128
│    └─ReLU: 2-3                              [160, 64, 56, 56]         --
│    └─MaxPool2d: 2-4                         [160, 64, 28, 28]         --
│    └─Sequential: 2-5                        [160, 256, 28, 28]        --
│    │    └─Bottleneck: 3-1                   [160, 256, 28, 28]        63,488
│    │    └─Bottleneck: 3-2                   [160, 256, 28, 28]        71,168
│    │    └─Bottleneck: 3-3                   [160, 256, 28, 28]        71,168
│    └─Sequential: 2-6                        [160, 512, 14, 14]        --
│    │    └─Bottleneck: 3-4                   [160, 512, 14, 14]        349,184